# 🤗 Welcome to AdalFlow with Ollama!
## Using Local LLMs with AdalFlow via Ollama

This tutorial demonstrates how to use Ollama with AdalFlow to run local LLMs like gpt-oss. Ollama allows you to run open-source models locally without depending on external APIs.

Thanks for trying us out! 😊 Any questions or concerns you may have, [come talk to us on discord,](https://discord.gg/ezzszrRZvT) we're always here to help! ⭐ <i>Star us on <a href="https://github.com/SylphAI-Inc/AdalFlow">Github</a> </i> ⭐

# Quick Links

Github repo: https://github.com/SylphAI-Inc/AdalFlow

Full Tutorials: https://adalflow.sylph.ai/index.html#.

Ollama Documentation: https://ollama.com/

# Author

This notebook was created by the AdalFlow team.

# Outline

This tutorial covers:

* Setting up Ollama locally
* Basic synchronous chat with AdalFlow Generator
* Asynchronous chat operations
* Streaming responses
* Configuring model parameters

# Prerequisites

1. Install Ollama from https://ollama.com/
2. Pull the model you want to use (e.g., `ollama pull llama2` or `ollama pull mistral`)
3. Ensure Ollama is running locally (default port: 11434)

## Installation

Install AdalFlow with Ollama support:

In [ ]:
from IPython.display import clear_output

!pip install -U adalflow ollama

clear_output()

In [1]:
import subprocess
import time
import requests
import os

def is_ollama_running():
    """Check if Ollama is running"""
    try:
        response = requests.get("http://localhost:11434/api/version", timeout=2)
        return response.status_code == 200
    except:
        return False

if not is_ollama_running():
    print("Starting Ollama server...")
    # Start Ollama in the background using subprocess.Popen
    # This won't block the notebook
    process = subprocess.Popen(
        ['ollama', 'serve'],
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL,
        start_new_session=True  # This detaches the process
    )

    # Wait a few seconds for Ollama to start
    for i in range(10):
        time.sleep(1)
        if is_ollama_running():
            print("✅ Ollama server started successfully!")
            break
    else:
        print("⚠️ Ollama might take longer to start. Please wait and re-run the next cell.")
else:
    print("✅ Ollama is already running!")


Starting Ollama server...
✅ Ollama server started successfully!


## Check Ollama Connection

First, let's verify that Ollama is running and accessible:

In [2]:
import requests

try:
    response = requests.get("http://localhost:11434/api/version")
    if response.status_code == 200:
        print("✅ Ollama is running!")
        print(f"Version: {response.json()}")
    else:
        print("❌ Ollama is not responding correctly")
except requests.exceptions.ConnectionError:
    print("❌ Cannot connect to Ollama. Please make sure Ollama is running.")
    print("Run 'ollama serve' in your terminal to start Ollama.")

✅ Ollama is running!
Version: {'version': '0.11.3'}


## List Available Models

Let's see what models are available in your Ollama installation:

In [2]:
import requests

response = requests.get("http://localhost:11434/api/tags")
if response.status_code == 200:
    models = response.json().get('models', [])
    if models:
        print("Available models:")
        for model in models:
            print(f"  - {model['name']} ({model['size'] / 1e9:.2f} GB)")
    else:
        print("No models found. Pull a model using: ollama pull llama2")
else:
    print("Could not fetch models")

Available models:
  - qwen2:0.5b (0.35 GB)
  - gpt-oss:20b (13.78 GB)


# 😇 Basic Usage with AdalFlow Generator

Let's start with the simplest way to use Ollama with AdalFlow's Generator component. We'll use the `gpt-oss:20b` model for these examples.

## Simple Text Generation

The Generator is AdalFlow's main component for interacting with language models. Here's how to use it with Ollama:

In [4]:
from adalflow.components.model_client.ollama_client import OllamaClient
from adalflow.core import Generator

# Initialize the Generator with OllamaClient
# Using gpt-oss:20b model as shown in the test file
generator = Generator(
    model_client=OllamaClient(host="http://localhost:11434"),
    model_kwargs={
        "model": "gpt-oss:20b",  # Using gpt-oss model
    }
)

# Test with a simple prompt
response = generator.call(prompt_kwargs={"input_str": "Hello! What are the benefits of using local LLMs"})
print("Response:")
print(response.data)


Response:
**Why run a Large Language Model (LLM) locally?**  
| Benefit | What it means in practice |
|---------|---------------------------|
| **Data privacy & security** | Your text never leaves your machine, so sensitive or proprietary information stays internal. No risk of data being logged by a third‑party service. |
| **Regulatory compliance** | For industries bound by GDPR, HIPAA, or other privacy laws, keeping data on‑premises helps meet audit and data‑handling requirements. |
| **Low latency** | Inference happens directly on your hardware, eliminating network round‑trips. Ideal for real‑time chatbots, on‑device assistants, or edge deployments. |
| **Offline availability** | No internet connection is needed once the model is downloaded. Useful for remote locations, airplanes, or environments with strict firewall rules. |
| **Cost control** | After the initial compute investment, you avoid recurring cloud‑API fees. You only pay for the hardware you own or rent. |
| **Customizabi

In [5]:
# thinking 

response.thinking

'The user asks: "Hello! What are the benefits of using local LLMs". They want benefits of local large language models. So we need to answer concisely and informatively. Should cover privacy, data security, latency, offline access, customizability, no dependency on cloud, cost control, compliance, etc. Also mention possible tradeoffs like requiring compute. Provide bullet points. The user may want a concise answer. Let\'s produce a friendly answer.'

## Asynchronous Call

For better performance, you can use the async version with `acall`:

In [6]:

# Using async call with acall
output = await generator.acall(prompt_kwargs={"input_str": "What are the advantages of async programming?"})

print("Async Response:")
print(output.data)

Async Response:
## Advantages of Asynchronous (Async) Programming

| # | Advantage | What It Means | Why It Matters |
|---|-----------|---------------|----------------|
| 1 | **Better Responsiveness** | The main thread never blocks on I/O or long‑running tasks. | UI apps stay snappy, servers keep handling new requests while previous ones are still in flight. |
| 2 | **Higher Throughput & Scalability** | A single thread can manage thousands of I/O operations simultaneously. | On servers you can handle many concurrent connections with far fewer OS threads or processes than with a thread‑per‑connection model. |
| 3 | **Reduced Resource Consumption** | Fewer threads → less memory, context‑switching, and scheduling overhead. | Critical for mobile devices, embedded systems, or data‑center cost savings. |
| 4 | **Simpler Error Handling & Composition** | `async/await`, `Task`, `Promise`, etc. let you write linear‑looking code that’s actually non‑blocking. | Avoids callback “pyramid of doom” an

## Streaming Responses

For real-time output, you can stream responses directly from Ollama:

In [6]:
# pull qwen2:0.5b

!ollama pull qwen2:0.5b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest 
pulling 8de95da68dc4:   0% ▕                  ▏  98 KB/352 MB                  pulling manifest 
pulling 8de95da68dc4:   0% ▕                  ▏ 299 KB/352 MB                  pulling manifest 
pulling 8de95da68dc4:   0% ▕                  ▏ 590 KB/352 MB                  pulling manifest 
pulling 8de95da68dc4:   0% ▕                  ▏ 1.4 MB/352 MB                  pulling manifest 
pulling 8de95da68dc4:   0% ▕                  ▏ 1.6 MB/352 MB                  pulling manifest 
pulling 8de95da68dc4:   1% ▕                  ▏ 2.3 MB/352 MB                  pulling manifest 
pulling 8de95da68dc4:   1% ▕                  ▏ 2.9 MB/352 MB                  pulling manifest 
pulling 8de95da68dc4:   1% ▕                  ▏ 3.4 MB/352 MB                  pulling manifest 
pulling 8de95da68dc4:   1% ▕       

In [5]:
from adalflow.components.model_client.ollama_client import OllamaClient
from adalflow.core import Generator

stream_generator = Generator(
    model_client=OllamaClient(host="http://localhost:11434"),
    model_kwargs={
        "model": "gpt-oss:20b",
        "stream": True,  # Enable streaming
    }
)

# async call with streaming
output = await stream_generator.acall(prompt_kwargs={"input_str": "Why is the sky blue?"})

async for chunk in output.raw_response:
    print(chunk["message"]["content"], end='', flush=True)

print(output)

api_kwargs: {'model': 'gpt-oss:20b', 'stream': True, 'messages': [{'role': 'user', 'content': '<START_OF_SYSTEM_PROMPT>\nYou are a helpful assistant.\n<END_OF_SYSTEM_PROMPT>\n<START_OF_USER_PROMPT>\nWhy is the sky blue?\n<END_OF_USER_PROMPT>\n'}]}
The sky looks blue because the molecules and very tiny particles in the Earth’s atmosphere scatter sunlight, and they scatter shorter‑wavelength light (the “blue” part of the spectrum) much more efficiently than longer wavelengths.

**Rayleigh scattering** – the physics behind it – says that when light encounters particles that are much smaller than its wavelength, the scattering intensity falls off with the fourth power of the wavelength. In practice this means:

- Blue light (≈ 400‑500 nm) is scattered 10–15 times more strongly than red light (≈ 600‑700 nm).
- The scattered blue photons are sent in all directions, so even the sky far from the sun appears bright blue.
- During sunrise or sunset the sun’s light passes through a longer atmosph

In [6]:
# sync Call with streaming
output = stream_generator.call(prompt_kwargs={"input_str": "Why is the sky blue?"})

for chunk in output.raw_response:
    print(chunk["message"]["content"], end='', flush=True)

print(output)

api_kwargs: {'model': 'gpt-oss:20b', 'stream': True, 'messages': [{'role': 'user', 'content': '<START_OF_SYSTEM_PROMPT>\nYou are a helpful assistant.\n<END_OF_SYSTEM_PROMPT>\n<START_OF_USER_PROMPT>\nWhy is the sky blue?\n<END_OF_USER_PROMPT>\n'}]}
The sky appears blue because of the way Earth’s atmosphere scatters sunlight.

### 1. Light from the Sun
Sunlight is actually a mix of all visible wavelengths (red, orange, yellow, green, blue, indigo, violet). Each color has a different wavelength: violet and blue are the shortest (≈400–495 nm), while red is the longest (≈620–750 nm).

### 2. Rayleigh scattering
- **What it is**: When sunlight hits the very small molecules and particles in the air (nitrogen, oxygen, water vapor, dust), the light is scattered in all directions.
- **Why shorter wavelengths scatter more**: Rayleigh scattering’s intensity is inversely proportional to the fourth power of the wavelength (\(I \propto 1/\lambda^4\)). This means a 450 nm blue photon scatters about 10

# Issues and Feedback

If you encounter any issues, please report them here: [GitHub Issues](https://github.com/SylphAI-Inc/AdalFlow/issues).

For feedback, you can use either the [GitHub discussions](https://github.com/SylphAI-Inc/AdalFlow/discussions) or [Discord](https://discord.gg/ezzszrRZvT).

For Ollama-specific issues, visit: [Ollama GitHub](https://github.com/ollama/ollama)